In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import requests
from config import keys

## Davidson Dataset

In [2]:
df_davidson = pd.read_csv("davidson.csv")
df_davidson

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [3]:
df_davidson = df_davidson.drop(columns=["Unnamed: 0", "count", "hate_speech", "offensive_language", "neither"],axis = 1)
df_davidson

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,2,"you've gone and broke the wrong heart baby, an..."
24780,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,1,youu got wild bitches tellin you lies


In [4]:
df_davidson.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   class   24783 non-null  int64 
 1   tweet   24783 non-null  object
dtypes: int64(1), object(1)
memory usage: 387.4+ KB


In [12]:
df_davidson["class"].nunique()

3

In [9]:
df_davidson["class"].value_counts()

1    19190
2     4163
0     1430
Name: class, dtype: int64

In [13]:
df_davidson = df_davidson.replace({"class": {1: 0, 2: 0, 0: 1}})
df_davidson["class"].value_counts()

0    23353
1     1430
Name: class, dtype: int64

## Aristotle Dataset

In [38]:
df_aristotle = pd.read_csv("aristotle.csv")
df_aristotle

,tweet_id,maj_label
0,849667487180259329,abusive
1,850490912954351616,abusive
2,848791766853668864,abusive
3,848306464892604416,abusive
4,850010509969465344,normal
...,...,...
79995,847640895956459521,normal
79996,850335795022102530,normal
79997,848201162717265920,normal
79998,848939985160077312,spam


In [39]:
df_aristotle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   80000 non-null  int64 
 1   maj_label  79996 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.2+ MB


In [42]:
df_aristotle.isna().sum()

tweet_id     0
maj_label    4
dtype: int64

In [53]:
df_aristotle[df_aristotle["maj_label"].isna()]

,tweet_id,maj_label
197,848898276984311808,NaN
3292,849950032274964480,NaN
4241,849964053833428994,NaN
15359,848040411797102592,NaN


In [59]:
df_aristotle = df_aristotle.dropna()

In [60]:
df_aristotle.isna().sum()

tweet_id     0
maj_label    0
dtype: int64

In [55]:
df_aristotle["maj_label"].value_counts()

normal     52835
spam       13404
abusive    10122
hateful     3635
Name: maj_label, dtype: int64

In [65]:
def create_batch(arr, size=50):
    final_arr =[]
    i = 0
    while i < len(arr):        
        final_arr.append(','.join([str(item) for item in arr[i:i+size]]))
        i += size
    return final_arr

In [66]:
def get_tweets(tweets_id):
    arr = []
    for batch in tweets_id:
        url = "https://api.twitter.com/2/tweets?ids={}".format(batch)
        res={}
        headers = {'Authorization': 'Bearer ' + keys['bearer_token']}
        r = requests.request("GET", url, headers=headers, data=res)
        
        data = r.json()
        if 'data' in data.keys():
            arr.append(pd.DataFrame(data['data']))
            
    return pd.concat(arr)

In [67]:
df_hate = df_aristotle[df_aristotle["maj_label"] == "hateful"]
hate_id = create_batch(list(df_hate.tweet_id))

In [68]:
df_hate = get_tweets(hate_id)

In [69]:
df_hate

,id,text
0,850433664890544128,I hate er chase because if the Bitch that work...
1,850100742035836929,RT @nyctophil3: Pineapples do not belong on pi...
2,847804507367100416,Niggas keep talking about women wearing weave ...
3,850090319165050880,@vappywave idiot that's not gonna work. you go...
4,848686686930382848,RT @ayevonnn: bruh i fucking hate people like ...
...,...,...
19,850701219568001025,Baekhyun won 'Most Popular Artist of Korea'\n\...
20,849316180670259205,#TripleTalaqArrest 2fgt dis social menace #Tri...
21,849674709763358720,White nationalists' latest tactic to recruit c...
22,849246676854484992,We are in a comfortable lead fight against gal...


In [70]:
df_hate = df_hate.drop(columns=["id"], axis=1)
df_hate

,text
0,I hate er chase because if the Bitch that work...
1,RT @nyctophil3: Pineapples do not belong on pi...
2,Niggas keep talking about women wearing weave ...
3,@vappywave idiot that's not gonna work. you go...
4,RT @ayevonnn: bruh i fucking hate people like ...
...,...
19,Baekhyun won 'Most Popular Artist of Korea'\n\...
20,#TripleTalaqArrest 2fgt dis social menace #Tri...
21,White nationalists' latest tactic to recruit c...
22,We are in a comfortable lead fight against gal...


In [94]:
df_hate = df_hate.rename(columns={"text": "tweet"})
df_hate["class"] = 1
df_hate

,tweet,class
0,I hate er chase because if the Bitch that work...,1
1,RT @nyctophil3: Pineapples do not belong on pi...,1
2,Niggas keep talking about women wearing weave ...,1
3,@vappywave idiot that's not gonna work. you go...,1
4,RT @ayevonnn: bruh i fucking hate people like ...,1
...,...,...
19,Baekhyun won 'Most Popular Artist of Korea'\n\...,1
20,#TripleTalaqArrest 2fgt dis social menace #Tri...,1
21,White nationalists' latest tactic to recruit c...,1
22,We are in a comfortable lead fight against gal...,1


In [95]:
df_hate = df_hate[["class", "tweet"]]
df_hate

,class,tweet
0,1,I hate er chase because if the Bitch that work...
1,1,RT @nyctophil3: Pineapples do not belong on pi...
2,1,Niggas keep talking about women wearing weave ...
3,1,@vappywave idiot that's not gonna work. you go...
4,1,RT @ayevonnn: bruh i fucking hate people like ...
...,...,...
19,1,Baekhyun won 'Most Popular Artist of Korea'\n\...
20,1,#TripleTalaqArrest 2fgt dis social menace #Tri...
21,1,White nationalists' latest tactic to recruit c...
22,1,We are in a comfortable lead fight against gal...


In [100]:
compression_opts = dict(method='zip',

                        archive_name='aristotle_hate.csv')  

df_hate.to_csv('aristotle_hate.zip', index=False,

          compression=compression_opts)  

## Copenhagen Dataset

In [102]:
df_copenhagen = pd.read_csv("University_of_Copenhagen.csv")
df_copenhagen

,572342978255048705,racism
0,572341498827522049,racism
1,572340476503724032,racism
2,572334712804384768,racism
3,572332655397629952,racism
4,575949086055997440,racism
...,...,...
16901,576359685843861505,none
16902,576612926838046720,none
16903,576771329975664640,none
16904,560595245814267905,none


In [103]:
df_copenhagen = df_copenhagen.rename(columns={"572342978255048705": "tweet_id", "racism": "class"})
df_copenhagen

,tweet_id,class
0,572341498827522049,racism
1,572340476503724032,racism
2,572334712804384768,racism
3,572332655397629952,racism
4,575949086055997440,racism
...,...,...
16901,576359685843861505,none
16902,576612926838046720,none
16903,576771329975664640,none
16904,560595245814267905,none


In [104]:
df_copenhagen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16906 entries, 0 to 16905
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet_id  16906 non-null  int64 
 1   class     16906 non-null  object
dtypes: int64(1), object(1)
memory usage: 264.3+ KB


In [106]:
df_copenhagen["class"].value_counts()

none      11559
sexism     3378
racism     1969
Name: class, dtype: int64

In [112]:
df_copenhate = df_copenhagen[(df_copenhagen["class"] == "sexism") | (df_copenhagen["class"] == "racism")]
df_copenhate

,tweet_id,class
0,572341498827522049,racism
1,572340476503724032,racism
2,572334712804384768,racism
3,572332655397629952,racism
4,575949086055997440,racism
...,...,...
5342,570244879265206272,sexism
5343,571013698111860736,sexism
5344,571304517930774528,sexism
5345,575664315627216896,sexism


In [113]:
copenhate_id = create_batch(list(df_copenhate.tweet_id))
df_copenhate = get_tweets(copenhate_id)

In [114]:
df_copenhate

,id,text
0,572341498827522049,Drasko they didn't cook half a bird you idiot ...
1,572340476503724032,Hopefully someone cooks Drasko in the next ep ...
2,572334712804384768,of course you were born in serbia...you're as ...
3,572332655397629952,These girls are the equivalent of the irritati...
0,446460991396917248,RT @YesYoureRacist: At least you're only a tin...
...,...,...
24,569990621315919872,"RT @Superjutah: @MT8_9 £34,000 is £9000 above ..."
25,570244879265206272,RT @DufresneFrank: “@gabystama: @DufresneFrank...
26,571013698111860736,RT @BoycottBrandy: @ActionFlickDoc this person...
27,571304517930774528,RT @Lyall: Patriarchy™ will expel me if I divu...


In [115]:
df_copenhate.drop(columns=["id"], axis=1, inplace=True)
df_copenhate

,text
0,Drasko they didn't cook half a bird you idiot ...
1,Hopefully someone cooks Drasko in the next ep ...
2,of course you were born in serbia...you're as ...
3,These girls are the equivalent of the irritati...
0,RT @YesYoureRacist: At least you're only a tin...
...,...
24,"RT @Superjutah: @MT8_9 £34,000 is £9000 above ..."
25,RT @DufresneFrank: “@gabystama: @DufresneFrank...
26,RT @BoycottBrandy: @ActionFlickDoc this person...
27,RT @Lyall: Patriarchy™ will expel me if I divu...


In [120]:
df_copenhate = df_copenhate.rename(columns={"text": "tweet"})
df_copenhate["class"] = 1
df_copenhate = df_copenhate[["class", "tweet"]]

In [121]:
df_copenhate.head()

,class,tweet
0,1,Drasko they didn't cook half a bird you idiot ...
1,1,Hopefully someone cooks Drasko in the next ep ...
2,1,of course you were born in serbia...you're as ...
3,1,These girls are the equivalent of the irritati...
0,1,RT @YesYoureRacist: At least you're only a tin...


In [125]:
compression_opts = dict(method='zip',

                        archive_name='copenhagen_hate.csv')  

df_copenhate.to_csv('copenhagen_hate.zip', index=False,

          compression=compression_opts)  

In [126]:
df_copenhate

,class,tweet
0,1,Drasko they didn't cook half a bird you idiot ...
1,1,Hopefully someone cooks Drasko in the next ep ...
2,1,of course you were born in serbia...you're as ...
3,1,These girls are the equivalent of the irritati...
0,1,RT @YesYoureRacist: At least you're only a tin...
...,...,...
24,1,"RT @Superjutah: @MT8_9 £34,000 is £9000 above ..."
25,1,RT @DufresneFrank: “@gabystama: @DufresneFrank...
26,1,RT @BoycottBrandy: @ActionFlickDoc this person...
27,1,RT @Lyall: Patriarchy™ will expel me if I divu...


## HASOC Dataset

In [144]:
df_hasoc = pd.read_csv("english_dataset.tsv", delimiter = "\t")
df_hasoc

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE
...,...,...,...,...,...
5847,hasoc_en_5848,"@davidfrum @trueblueusa1 That's cute and all, ...",HOF,PRFN,UNT
5848,hasoc_en_5849,a recession issa comin' #maga #magamyass #fuck...,NOT,NONE,NONE
5849,hasoc_en_5850,#DoctorsFightBack Will 'The Mad n Irrational ...,HOF,OFFN,TIN
5850,hasoc_en_5851,#ShiningIndia #educatedindia or more like RUND...,HOF,OFFN,UNT


In [145]:
df_hasoc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5852 entries, 0 to 5851
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text_id  5852 non-null   object
 1   text     5852 non-null   object
 2   task_1   5852 non-null   object
 3   task_2   5852 non-null   object
 4   task_3   5852 non-null   object
dtypes: object(5)
memory usage: 228.7+ KB


In [146]:
df_hasoc["task_2"].value_counts()

NONE    3591
HATE    1143
PRFN     667
OFFN     451
Name: task_2, dtype: int64

In [147]:
df_hasoc.drop(columns=["text_id", "task_1", "task_3"], axis=1, inplace=True)
df_hasoc = df_hasoc.rename(columns={"text": "tweet", "task_2": "class"})

In [149]:
df_hasoc = df_hasoc[df_hasoc["class"] == "HATE"]

In [154]:
df_hasoc["class"] = 1
df_hasoc = df_hasoc[["class", "tweet"]]

In [155]:
df_hasoc

,class,tweet
1,1,@politico No. We should remember very clearly ...
11,1,I don’t know how much more I can take! 45 is a...
23,1,#ShameOnICC 1. ICC on Dhoni's gloves ...
27,1,Disgusting! And they don’t change their cultur...
34,1,"@TheRealOJ32 Of all the retired NFL players, w..."
...,...,...
5840,1,Full of fools.. ICC @ICCMediaComms @ICC @crick...
5842,1,"The US-Mexican border, where established moral..."
5844,1,@ICC All Eng matches are on dry pitches &amp; ...
5845,1,It seems the Rain &amp; thunder god only like ...


## Creating Final Dataset

In [156]:
df_final = pd.concat([df_davidson, df_hate, df_copenhate, df_hasoc], ignore_index=True)
df_final.head()

,class,tweet
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [157]:
df_final["class"].value_counts()

0    23353
1     6988
Name: class, dtype: int64

In [158]:
compression_opts = dict(method='zip',

                        archive_name='final_dataset.csv')  

df_final.to_csv('final_dataset.zip', index=False,

          compression=compression_opts)